In [ ]:
!pip install ultralytics

In [ ]:
!pip install mediapipe

In [ ]:
!pip install cvzone

In [ ]:
import os
import cv2
import pandas as pd
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

# Load the YOLO model
model = YOLO('yolov8n.pt')

# Define directories for multiple folders
base_dirs = [
    '/content/drive/MyDrive/data_set1/data_set1/Coffee/Annotations_files',
    '/content/drive/MyDrive/data_set1/data_set1/Coffee/Video',
    '/content/drive/MyDrive/data_set1/data_set1/Home/Annotations_files',
    '/content/drive/MyDrive/data_set1/data_set1/Home/Video'
]

# Output directory in Google Drive
output_base_dir = '/content/drive/MyDrive/Fall_detectionp'  # Main output directory in Google Drive
fall_dir = os.path.join(output_base_dir, 'Fall')
non_fall_dir = os.path.join(output_base_dir, 'Non_Fall')

# Create the directories for Fall and Non-Fall if they don't exist
os.makedirs(fall_dir, exist_ok=True)
os.makedirs(non_fall_dir, exist_ok=True)

# Initialize a list to hold the frame filenames, labels, and bounding box coordinates
data = []

# Confidence threshold for detection
confidence_threshold = 0.5

# Aspect ratio threshold for fall detection
aspect_ratio_threshold = 1  # Updated to 1.5 for better classification

# Process videos
for base_dir in base_dirs:
    video_files = [f for f in os.listdir(base_dir) if f.endswith('.avi')]

    print(f"Found video files in {base_dir}: {video_files}")

    for video_file in video_files:
        video_path = os.path.join(base_dir, video_file)
        cap = cv2.VideoCapture(video_path)
        frame_counter = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_counter += 1  # Increment frame counter for every frame processed

            # Detect objects using the YOLO model
            results = model(frame)
            results = results[0]

            detected = False
            for box in results.boxes:
                # Extract bounding box coordinates and dimensions
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = box.conf[0].item()  # Confidence score
                cls = int(box.cls[0].item())  # Class ID (0 = person)

                # Process only if the detected class is 'person' and confidence is high enough
                if cls == 0 and conf > confidence_threshold:
                    detected = True

                    # Compute bounding box dimensions
                    width = x2 - x1
                    height = y2 - y1

                    # Debugging: Print dimensions and aspect ratio
                    print(f"Frame {frame_counter}: Detected 'person' with confidence {conf:.2f}")
                    print(f"Bounding Box: x1={x1}, y1={y1}, x2={x2}, y2={y2}, Width={width}, Height={height}")

                    # Calculate the aspect ratio
                    aspect_ratio = height / width
                    print(f"Aspect Ratio: {aspect_ratio:.2f} (Threshold: {aspect_ratio_threshold})")

                    # Classify Fall vs. Non-Fall based on aspect ratio
                    if aspect_ratio < aspect_ratio_threshold:  # Fall condition
                        label = 'Fall'
                        color = (0, 0, 255)  # Red for Fall
                        save_dir = fall_dir
                    else:  # Non-Fall condition
                        label = 'Non-Fall'
                        color = (0, 255, 0)  # Green for Non-Fall
                        save_dir = non_fall_dir

                    # Draw bounding box and label
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    text = f"{label} ({conf:.2f})"
                    cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                    # Save the frame
                    frame_filename = os.path.join(save_dir, f"{video_file}_frame_{frame_counter:04d}_{label}.jpg")
                    cv2.imwrite(frame_filename, frame)

                    # Append to data (add bounding box coordinates)
                    data.append([frame_filename, label, x1, y1, x2, y2])
                    print(f"Saved {label} frame: {frame_filename}")
                    break  # Process only one bounding box per frame

            # Display the frame (for visual debugging in Colab)
            cv2_imshow(frame)

            # Break loop for testing (remove this for full processing)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()

# Save the results to a CSV file
labels_df = pd.DataFrame(data, columns=['image_path', 'label', 'x1', 'y1', 'x2', 'y2'])
labels_df.to_csv(os.path.join(output_base_dir, 'labels.csv'), index=False)

cv2.destroyAllWindows()
